teoria actividad 3 semana 7

In [ ]:
#ANOVA de Una Vía y su Interpretación

"""
Asunciones del ANOVA
Independencia: Observaciones independientes entre grupos.
Normalidad: Distribuciones aproximadamente normales en cada grupo.
Homocedasticidad: Varianzas iguales entre grupos (prueba de Levene).
"""

In [2]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np

# Placeholder for 'grupos'. Replace this with your actual data.
# 'grupos' should be a dictionary where keys are group names and values are arrays/lists of data.
grupos = {
    'Grupo A': np.random.normal(loc=10, scale=2, size=30),
    'Grupo B': np.random.normal(loc=12, scale=2, size=30),
    'Grupo C': np.random.normal(loc=10, scale=2, size=30)
}

# Verificar normalidad (Shapiro-Wilk por grupo)
for grupo, datos in grupos.items():
    stat, p = stats.shapiro(datos)
    print(f"Grupo {grupo} - Normal: {'Sí' if p > 0.05 else 'No'} (p={p:.3f})")

# Verificar homocedasticidad (Levene)
stat, p = stats.levene(*grupos.values())
print(f"Homocedasticidad: {'Sí' if p > 0.05 else 'No'} (p={p:.3f})")

Grupo Grupo A - Normal: Sí (p=0.991)
Grupo Grupo B - Normal: Sí (p=0.694)
Grupo Grupo C - Normal: Sí (p=0.847)
Homocedasticidad: Sí (p=0.636)


In [3]:
# ANOVA de una vía
f_stat, p_value = stats.f_oneway(*grupos.values())

print(f"Estadístico F: {f_stat:.3f}")
print(f"Valor p: {p_value:.4f}")
print(f"Significativo: {'Sí' if p_value < 0.05 else 'No'}")

Estadístico F: 7.151
Valor p: 0.0013
Significativo: Sí


In [5]:
import statsmodels.formula.api as smf
import pandas as pd

# Preparar datos en formato long
df_long = pd.DataFrame({
    'valor': np.concatenate(list(grupos.values())),
    'grupo': np.repeat(list(grupos.keys()), [len(v) for v in grupos.values()])
})

# Modelo ANOVA
modelo = smf.ols('valor ~ C(grupo)', data=df_long).fit()
anova_table = sm.stats.anova_lm(modelo, typ=2)

print(anova_table)

              sum_sq    df         F    PR(>F)
C(grupo)   48.577825   2.0  7.151238  0.001332
Residual  295.492250  87.0       NaN       NaN


In [6]:
# Pruebas Post-hoc y Corrección de Bonferroni
# Prueba de Tukey
tukey = pairwise_tukeyhsd(df_long['valor'], df_long['grupo'], alpha=0.05)
print(tukey)

# Resultado típico:
# Multiple Comparison of Means - Tukey HSD, FWER=0.05
# ====================================================
# group1 group2 meandiff p-adj   lower   upper  reject
# ----------------------------------------------------
#   A      B     5.234   0.012  1.456   9.012   True
#   A      C     2.123   0.245 -1.234   5.480  False
#   B      C    -3.111   0.034 -6.234  -0.988   True

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower   upper  reject
------------------------------------------------------
Grupo A Grupo B   1.5195 0.0055  0.3848  2.6541   True
Grupo A Grupo C  -0.0753 0.9863   -1.21  1.0593  False
Grupo B Grupo C  -1.5948 0.0034 -2.7294 -0.4601   True
------------------------------------------------------


In [7]:
from statsmodels.sandbox.stats.multicomp import multipletests

# Aplicar corrección Bonferroni
p_values = [0.03, 0.04, 0.02]  # p-values de comparaciones individuales
reject, p_adjusted, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')

print("Corrección Bonferroni:")
for i, (p_orig, p_adj, rej) in enumerate(zip(p_values, p_adjusted, reject)):
    print(f"Comparación {i+1}: p={p_orig:.3f} → p-ajustado={p_adj:.3f} (Rechazar: {rej})")

Corrección Bonferroni:
Comparación 1: p=0.030 → p-ajustado=0.090 (Rechazar: False)
Comparación 2: p=0.040 → p-ajustado=0.120 (Rechazar: False)
Comparación 3: p=0.020 → p-ajustado=0.060 (Rechazar: False)


In [ ]:
#Analisis
"""
Objetivo del análisis

Determinar si existen diferencias significativas entre las medias de tres grupos:
Grupo A
Grupo B
Grupo C
👉 Cuando hay más de dos grupos, no se usan t-tests múltiples, sino ANOVA.



1-Verificación de supuestos del ANOVA: Antes de aplicar ANOVA, se deben cumplir ciertos supuestos.

1.1 Normalidad (Shapiro-Wilk)
stats.shapiro(datos)

Hipótesis:

H₀: los datos siguen una distribución normal

H₁: los datos no son normales

Resultados:

Grupo A p=0.991
Grupo B p=0.694
Grupo C p=0.847


👉 Todos los p-values > 0.05
✅ No se rechaza la normalidad




1.2 Homocedasticidad (Levene)
stats.levene(*grupos.values())

Hipótesis:

H₀: las varianzas de los grupos son iguales

H₁: al menos una varianza es distinta

Resultado:

p = 0.636 > 0.05


✅ Se cumple el supuesto de igualdad de varianzas


2-ANOVA de una vía
stats.f_oneway(*grupos.values())

🔹 Hipótesis

H₀: μ_A = μ_B = μ_C

H₁: al menos una media es distinta

🔹 Resultado
F = 7.151
p = 0.0013


📌 Como p < 0.05:
➡️ Se rechaza H₀

👉 Existe evidencia de diferencias entre los grupos, pero ANOVA no dice entre cuáles.




3-ANOVA con statsmodels (confirmación)
sm.stats.anova_lm(modelo, typ=2)

Fuente	  F	        p
Grupo	  7.151    	0.001332

👉 Confirma exactamente el resultado del ANOVA de SciPy.

4-Pruebas post-hoc: Tukey HSD
🔹 ¿Por qué post-hoc?: Porque ANOVA solo responde: ¿hay diferencias?

No responde:¿entre qué grupos?

🔹 Tukey HSD
pairwise_tukeyhsd(...)


Controla el error familiar (FWER) automáticamente.

🔹 Interpretación
Comparación   	p-ajustada    ¿Diferencia?
A vs B          	0.0055	      ✅ Sí
A vs C	          0.9863	      ❌ No
B vs C	          0.0034      	✅ Sí

📌 Grupo B es distinto de A y C
📌 A y C no difieren


5-Corrección de Bonferroni (concepto)
multipletests(..., method='bonferroni')

🔹 ¿Qué problema corrige?

Cuando haces muchas pruebas, aumenta la probabilidad de:

cometer al menos un error tipo I (falso positivo)
🔹 Cómo funciona Bonferroni
α_ajustado = α / número de pruebas


Con α = 0.05 y 3 comparaciones:

α_ajustado = 0.0167   (0.05/3)

🔹 Resultado
p original	p ajustado	¿Rechaza?
0.03          	0.09	     ❌
0.04          	0.12	     ❌
0.02	          0.06	     ❌

Ninguna comparación es significativa tras Bonferroni


6-Tukey vs Bonferroni (clave para entender)

Método	                 Característica

Tukey	                  Diseñado específicamente para comparar todas las medias
Bonferroni	            Muy conservador
Bonferroni	            Aumenta errores tipo II
Tukey	                  Mejor balance para ANOVA

👉 Por eso Tukey es preferido tras ANOVA.


7-Conclusión global

Los supuestos del ANOVA se cumplen

El ANOVA indica diferencias significativas

Tukey muestra que:

Grupo B es diferente

Grupos A y C son similares

Bonferroni ilustra cómo las correcciones afectan la significancia

🧠 Resumen mental para estudiar

Forma correcta de decirlo

ANOVA indica si existen diferencias globales entre los grupos.
Las pruebas post-hoc (como Tukey HSD) identifican entre qué grupos están esas diferencias.
Las correcciones por comparaciones múltiples (como Bonferroni) controlan el riesgo de falsos positivos (error tipo I).

🧠 Versión aún más corta (tipo memoria)
ANOVA detecta diferencias, Tukey dice dónde están y Bonferroni evita falsos positivos.

"""
